In [16]:
Output = '/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1007. Sputum Proteomics/P1007.3. Analyses/P1007.3.2. Distribution Tests/Output'
cur_date = "090924"

library(readxl)
library(tidyverse)
library(reshape2)

#reading in file
demographics_df = data.frame(read_excel("SubjectInfo_113022.xlsx", sheet = 2))
proteomics_df = data.frame(read_excel("011023_Processed_Proteomics_Data.xlsx"))
head(demographics_df)
head(proteomics_df)

,Original_Subject_Number,Proteomic_Core_Subject_Number,Proteomic_Core_Batch_Number,Subject_Number,Subject_ID,Sex,Race,Ethnicity,Age,BMI,Asthma_Status,Pre_Sputum_Neutrophilia,Post_Sputum_Neutrophilia,Responder_Status
,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>
1,1,1,2,1,NR_1,M,B,NH,37,26.0,NAS,77.20,74.49,NR
2,6,3,2,2,NR_2,F,W,NH,26,21.9,NAS,22.60,0.00,NR
3,11,9,1,3,NR_3,M,W,NH,33,23.9,NAS,68.39,0.00,NR
4,16,15,3,4,NR_4,F,W,NH,30,23.3,NAS,34.63,22.72,NR
5,23,23,1,5,NR_5,M,As,NH,29,19.6,NAS,28.60,79.70,NR
6,28,29,3,6,NR_6,F,W,NH,31,21.1,NAS,50.00,0.00,NR


,Protein_Accession,Gene_Name,Protein_Descriptions,Protein_Name,Number_Unique_Peptides,Subject_ID,Sample_ID,Intensity,Imputed_Norm_Intensity,Subject_No,PrevPost
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_1,Post_1,64819128,73788697,1,Post
2,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_1,Pre_2,88131664,81867431,1,Pre
3,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_2,Post_3,30396704,32521079,2,Post
4,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR_2,Pre_4,38267072,69466046,2,Pre
5,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R_16,Post_5,68394088,67261697,16,Post
6,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R_16,Pre_6,50484084,54090668,16,Pre


In [17]:
proteomics_df = proteomics_df %>%
    inner_join(demographics_df[,4:11]) %>%
    # just want a column that specifies the treatment and exposure separately
  separate(Subject_ID, c("R_NR", "ID"), sep = "_") %>%   
  separate(Sample_ID, c("Time_Point", "ID_2"), sep = "_") 

head(proteomics_df)

Joining with `by = join_by(Subject_ID)`


,Protein_Accession,Gene_Name,Protein_Descriptions,Protein_Name,Number_Unique_Peptides,R_NR,ID,Time_Point,ID_2,Intensity,Imputed_Norm_Intensity,Subject_No,PrevPost,Subject_Number,Sex,Race,Ethnicity,Age,BMI,Asthma_Status
,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<chr>
1,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,1,Post,1,64819128,73788697,1,Post,1,M,B,NH,37,26.0,NAS
2,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,1,Pre,2,88131664,81867431,1,Pre,1,M,B,NH,37,26.0,NAS
3,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,2,Post,3,30396704,32521079,2,Post,2,F,W,NH,26,21.9,NAS
4,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,NR,2,Pre,4,38267072,69466046,2,Pre,2,F,W,NH,26,21.9,NAS
5,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R,16,Post,5,68394088,67261697,16,Post,16,M,W,NH,40,29.8,NAS
6,P61626,LYZ,Lysozyme C,LYSC_HUMAN,21,R,16,Pre,6,50484084,54090668,16,Pre,16,M,W,NH,40,29.8,NAS


In [18]:
# creating additional dataframes that are subsetted by sex and asthma status
sex_df = proteomics_df %>%
    group_by(Sex) %>%
    group_split()

females_df = sex_df[[1]]
males_df = sex_df[[2]]

asthma_df = proteomics_df %>%
    group_by(Asthma_Status) %>%
    group_split()

asthmatic_df = asthma_df[[1]]
nonasthmatic_df = asthma_df[[2]]

In [24]:
proteins = unique(proteomics_df$Protein_Accession)

values_df = data.frame()
#create an empty dataframe for the loop function to place values into

timepoint_t_tests = function(dataframe, stratification){
    
    for (i in 1:length(proteins)){
        
         # pre df
        pre_df = dataframe %>%
            filter(Time_Point == "Pre", Protein_Accession == proteins[i])
        # post df
        post_df = dataframe %>%
            filter(Time_Point == "Post", Protein_Accession == proteins[i])
    
        # t test
        t_test = t.test(pre_df$Imputed_Norm_Intensity, post_df$Imputed_Norm_Intensity)
        
        # calculating log2FC to get directionality
        FC = log2(mean(pre_df$Imputed_Norm_Intensity)/mean(post_df$Imputed_Norm_Intensity))
    
        # contains stratification, protein, u stat, and p value
        values_vector = cbind(stratification, proteins[i], FC, t_test$statistic, t_test$p.value)
        values_df = rbind(values_df, values_vector)
    }    
    
    colnames(values_df) = c("Stratification", "Protein_Accession", "log2FC", "Statistic", 'P Value')
    values_df$`P Adj` <- p.adjust(values_df$`P Value`, method = "fdr")

    return(values_df)
}

# calling fn
timepoint_results_df = timepoint_t_tests(proteomics_df, "None")
males_results_df = timepoint_t_tests(males_df, "Male")
females_results_df = timepoint_t_tests(females_df, "Female")
asthmatic_results_df = timepoint_t_tests(asthmatic_df, "Asthmatic")
nonasthmatic_results_df = timepoint_t_tests(nonasthmatic_df, "Nonasthmatic")

In [25]:
# creating 1 df
all_results_df = rbind(timepoint_results_df, males_results_df, females_results_df, asthmatic_results_df, nonasthmatic_results_df)

head(all_results_df)

,Stratification,Protein_Accession,log2FC,Statistic,P Value,P Adj
,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
t,None,P61626,0.0728486146255376,0.585411887241969,0.560815096066013,0.7958692
t1,None,P01877,-0.212111161818743,-1.17593644942288,0.245514975976604,0.6021921
t2,None,P01876,-0.148517136427489,-0.982069487757299,0.33080863361061,0.6587227
t3,None,P02768,0.16409331368835,0.678951578641631,0.500869057582336,0.7684498
t4,None,P01834,-0.0149722167244042,-0.108493302294183,0.914024918562897,0.9551508
t5,None,Q8TDL5,0.132675814878414,0.590566484071262,0.557377293224984,0.7958692
